# 二维翼型强迫运动的模拟（贴体网格）


本教程将在[二维翼型定常流仿真模拟](./NACA_ALE_CASE1_STEADY_README.ipynb)的基础上进行翼型起伏运动 (Heaving) 的模拟，

\begin{equation}
x(t) = x(0) + (1 - e^{-(2\pi f t)^2}) \sin(2\pi f t) u
\end{equation}

我们需要指定频率 $f$ 和方向 $u$。**AERO-F**也支持翼型俯仰运动（Pitching）和指定形变的运动（Deforming 或者 Velocity），对于指定形变的运动我们需要给定**C++** 文件描述的形变函数。

我们将用[二维翼型定常流仿真模拟](./NACA_ALE_CASE1_STEADY_README.ipynb)的定常态作为流体初始状态开始模拟，也会用同样网格和网格分区，因此我们不需要再做前处理。

## 流体计算

流体计算的所有信息都在**AERO-F**的输入文件里。对于本次模拟，这个输入文件是 [simulations/case1/FluidFile.Forced](./FluidFile.Forced)。在这个文件夹里，我们一般有一些子文件夹：

- `references`用于储存模拟中途的流体状态，用于重启（Restart）模拟；
- `results`用于储存模拟中途的输出；
- `postpro`用于最后后处理过后的模拟结果。

**AERO-F**的输入文件分为许多个模块，值得一提的是，模块之间的顺序以及模块里指定的参数的顺序都没有任何限制。接下来，我们具体讲解本次仿真模拟的输入文件。


这里我们首先指定求解问题的类型，本次仿真使用贴体网格（BodyFitted），是无量纲的（NonDimensional）强迫运动（Forced）的模拟。

```shell
under Problem {
  Type = Forced;
  Mode = NonDimensional;
  Framework = BodyFitted;
}
```

然后指定网格相关的文件（Input），它们是网格分区预处理的生成文件，一般在`data`文件夹里面。这里可以给出这些网格相关文件的前缀，**AERO-F**会自动去查找这些文件。他们包括格点信息 `.msh`、网格分区标注信息 `.dec`, 网格连接信息 `.con`、CPU和分区映射的信息 `.cpu`，如果需要使用依赖于壁面距离的湍流模型（比如Spalart Allmaras模型）也需要给出预先计算好的流体格点到壁面的距离信息 `.dwall`。值得注意的是，我们这里也指定了流体的初始状态（Solution）是定常流模拟的结果。常规的重启模拟也需要重启数据文件（RestartData），包括了当前的时刻，由于本案例是重新开始模拟，因此不需要重启数据文件。

```shell
under Input {
  GeometryPrefix = "../../data/OUTPUT";
  
  Solution = "references.Steady/Solution.bin";
  //RestartData = "references.Steady/Restart.data";
}
```

然后指定仿真模拟中间的输出。这里包括我们感兴趣的一些量（Postpro）和用于重启模拟的文件（Restart），其中频率（Frequency）指定每多少步输出一次，如果频率是0，只在最后模拟结束时输出。

```shell
under Output {
  under Postpro {
    Frequency = 100;
    Prefix = "results.Forced/";
    LiftandDrag = "../postpro.Steady/LiftandDrag.out";
    Pressure = "Pressure.bin";
    Mach = "Mach.bin";
    PressureCoefficient = "PressureCoefficient.bin";
    SkinFriction = "SkinFriction.bin";
  }
  under Restart {
    Frequency = 0;
    Prefix = "references.Forced/";
    Solution = "Solution.bin";
    RestartData = "Restart.data";
    WallDistance = "WallDistance.bin";
  }
}
```

然后指定边界条件。这里包括远场边界条件（Inlet）和固体表面边界条件（Wall）。

```shell
under BoundaryConditions {
  under Inlet {
    Type = External;
    Mach = 0.4;
    Alpha = 0.0;
    Beta = 5.0;
  }
  under Wall {
    Type = Adiabatic;
    Integration = Full;
  }
}
```

由于是无量纲模拟，我们也需要指定参考状态，尤其是雷诺数。

```shell
under ReferenceState { 
  Reynolds = 10000;
  Length = 1.0;
}
```

然后指定我们要求解的方程以及相关的参数，比如Euler方程或者Navier Stokes方程，以及湍流闭包模型。

```shell
under Equations {  
  Type = NavierStokes;   
  under FluidModel[0] {
    Fluid = PerfectGas;
    under GasModel {
      SpecificHeatRatio = 1.4;
    }
  }
  under ThermalConductivityModel {
    Type = ConstantPrandtl;
  }
  under ViscosityModel {
    Type = Constant;
  }
  under TurbulenceClosure {
    Type = TurbulenceModel;
    under TurbulenceModel {
      Type = SpalartAllmaras;
    }
  }
}
```

然后指定空间离散格式。

```shell
under Space {
  under NavierStokes {
    Flux = Roe;
    Reconstruction = Linear;
    AdvectiveOperator = FiniteVolume;
    Limiter = None;
    Gradient = LeastSquares;
    Dissipation = SecondOrder;
    Beta = 0.3333333333333333;
    Gamma = 1.0;
  }
  under Boundaries {
    Type = StegerWarming;
  }
}
```

然后指定强迫运动，这里我们选择的是起伏运动，值得注意的是这里我们会指定时间步长，因此整个模拟是固定时间步长的，如果时间步长太大，流体求解时在每一步会自动迭代。

```shell
under Forced{
  under Heaving{
    AY = 0.2;
    AX = 0.0;
    AZ = 0.0;
  }
  Type = Heaving;
  Frequency = 0.15915494309; // 1/(2 * pi); 
  TimeStep = 1.e-2;
}
```

最后指定时间离散格式，包括CFL条件，显、隐格式，牛顿迭代的参数等。

```shell
under Time {
  MaxTime = 10.0;
  MaxIts = 10000;
  under CflLaw {
    Strategy = Fixed;
    Cfl0 = 100.0;
  }
  CheckLinearSolver = Off;
  Type = Implicit;
  under Implicit {
    Type=RungeKutta2;
    MatrixVectorProduct = FiniteDifference;
    TurbulenceModelCoupling = Weak;
    under Newton {
      MaxIts = 1;
      Eps = 0.01;
      FailSafe = Off;
      under LinearSolver {
        under NavierStokes {
          Type = Gmres;
          MaxIts = 50;
          KrylovVectors = 50;
          Eps = 0.01;
          under Preconditioner {
            Type = Ras;
            Fill = 0;
          }
        }
        under TurbulenceModel {
          Type = Gmres;
          MaxIts = 30;
          KrylovVectors = 30;
          Eps = 0.01;
          under Preconditioner {
            Type = Ras;
            Fill = 0;
          }
        }
      }
    }
  }
}
```

我们可以在`simulations/case1`中，用以下命令行在4个核上运行**AERO-F**。

```shell
mpirun -N 4 aerof.opt FluidFile.Forced
```

我们感兴趣的一些量，比如流体的压强（比如Pressure.bin1）马赫数（Mach.bin1）和压力系数（PresureCoefficient）都存在 `results.Forced`文件夹, 像压强和马赫数这些流体状态的量，都是每一个分区上的数据，因此各有4个文件。值得注意的是像压力系数这样的状态量，只存在于翼型表面的格点。像升力阻力（LiftandDrag）这些不需要后处理的量，我们存在了 `postpro.Forced`文件夹。
另外一些用于重启模拟的文件，都存在`references.Forced`文件夹。


## 结果后处理

由于**AERO-F**的并行，是每个CPU在各自分配到的划分的网格上进行计算，输出结果也是划分网格上的结果，我们需要首先用**sower**把输出结果拼接起来。我们可以在`simulations/case1`中，用以下命令行拼接关于压强、马赫数、压强系数和翼型表面摩擦的结果。

```shell
sower -fluid -merge -con ../../data/OUTPUT.con -mesh ../../data/OUTPUT.msh \
	-result results.Forced/Mach.bin -output postpro.Forced/Mach
    
sower -fluid -merge -con ../../data/OUTPUT.con -mesh ../../data/OUTPUT.msh \
	-result results.Forced/Pressure.bin -output postpro.Forced/Pressure
    
sower -fluid -merge -con ../../data/OUTPUT.con -mesh ../../data/OUTPUT.msh \
	-result results.Forced/PressureCoefficient.bin -output postpro.Forced/PressureCoefficient

sower -fluid -merge -con ../../data/OUTPUT.con -mesh ../../data/OUTPUT.msh \
	-result results.Forced/SkinFriction.bin -output postpro.Forced/SkinFriction
```

这里的输入包括网格分区的信息，即在`data`文件夹中各个分区的连接信息`OUTPUT.con`和各区域的网格信息（比如`OUTPUT.msh1`），以及模拟输出在`results.Steady`文件夹中的信息（比如`Mach.bin`）。结果会存放在`postpro.Steady`文件夹里面，分别是 `Mach.xpost` 、 `Pressure.xpost`、`PressureCoefficient.xpost` 和 `SkinFriction.xpost`。 

由于拼接起来的文件是后缀为`.xpost`的文件，为了使用**paraview**做后处理，我们需要用**xp2exo**把后缀为`.xpost`的文件转化为后缀为`.exo`的文件, 我们可以在`simulations/case1`中，用以下命令行进行转换

```shell
xp2exo ../../sources/fluid.top fluid_solution.Forced.exo \
                   ../../sources/fluid.top.dec.${NF} \
                   postpro.Forced/Mach.xpost postpro.Forced/Pressure.xpost \
                   postpro.Forced/PressureCoefficient.xpost postpro.Forced/SkinFriction.xpost
```

这里的输入包括在`sources`文件夹中的流体网格`fluid.top`和流体分区的信息`fluid.top.dec.4`，以及**sower**拼接后的我们感兴趣的流体各个量（包括`Mach.xpost`和`Pressure.xpost`，最终结果会存放在`fluid_solution.Forced.exo`面。


我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入`fluid_solution.exo`进行可视化。
这里我们展示在时刻 $t=10$ 的时候的马赫场
<img src="../../../../Figs/NACA_ALE_SA_Forced.png" width="500" />


## bash脚本运行

对于运行**AERO-F**以及结果的后处理，我们可以这些命令行写在`simulations/case1`文件夹里的脚本文件里，比如 [run.Forced.sh](./run.Forced.sh)。我们可以用以下命令运行脚本

```shell
bash run.Forced.sh
```

为了在集群上提交作业，我们需要用 [sbatch命令](https://hpc.pku.edu.cn/_book/guide/slurm/sbatch.html) 将作业提交到计算节点上执行，在 `simulations/case1` 文件夹里，我们可以编写作业脚本文件 [NACA.ALE.Case1.Forced.sh](./NACA.ALE.Case1.Forced.sh) 申请计算节点，进行计算。可以用以下命令行提交作业

```shell
sbatch NACA.ALE.Case1.Forced.sh
```



